# Packages and data imports

In [13]:
###########
# IMPORTS #
###########

%pylab inline --no-import-all

import os
from pathlib import Path

import pandas as pd
import numpy as np 

# Change this path to adapt to where you downloaded the data
DATA_PATH = Path("./geolifeclef-2022-lifeclef-2022-fgvc9")

# Create the path to save submission files
SUBMISSION_PATH = Path("./submissions")
os.makedirs(SUBMISSION_PATH, exist_ok=True)

# Clone the GitHub repository
# !rm -rf GLC
# !git clone https://github.com/maximiliense/GLC
    
    
# For evaluation and submission
from GLC.metrics import top_30_error_rate, top_k_error_rate_from_sets, predict_top_30_set
from GLC.submission import generate_submission_file

# For data loading and visualization
from GLC.data_loading.common import load_patch
from GLC.plotting import visualize_observation_patch
from GLC.data_loading.environmental_raster import PatchExtractor



################
# DATA LOADING #
################
print("Observations loading")

# Load train set of observations from France and USA and merge
df_obs_fr = pd.read_csv(DATA_PATH / "observations" / "observations_fr_train.csv", sep=";", index_col="observation_id")
df_obs_us = pd.read_csv(DATA_PATH / "observations" / "observations_us_train.csv", sep=";", index_col="observation_id")
df_obs = pd.concat((df_obs_fr, df_obs_us))

# Extract training and validation subsets as np arrays
obs_id_train = df_obs.index[df_obs["subset"] == "train"].values
obs_id_val = df_obs.index[df_obs["subset"] == "val"].values

# Separate values to predict
y_train = df_obs.loc[obs_id_train]["species_id"].values
y_val = df_obs.loc[obs_id_val]["species_id"].values

# Validation proportion
n_val = len(obs_id_val)
print("Validation set size: {} ({:.1%} of train observations)".format(n_val, n_val / len(df_obs)))


# Same with test set of observations
df_obs_fr_test = pd.read_csv(DATA_PATH / "observations" / "observations_fr_test.csv", sep=";", index_col="observation_id")
df_obs_us_test = pd.read_csv(DATA_PATH / "observations" / "observations_us_test.csv", sep=";", index_col="observation_id")
df_obs_test = pd.concat((df_obs_fr_test, df_obs_us_test))

# Extract observaions as np array
obs_id_test = df_obs_test.index.values

# Test set size
print("Number of observations for testing: {}".format(len(df_obs_test)))

# Display head of the df
print("Train df shape: ", df_obs.shape)
display(df_obs.head(3))
print("Test df shape: ", df_obs_test.shape)
display(df_obs_test.head(3))

Populating the interactive namespace from numpy and matplotlib
Observations loading
Validation set size: 40080 (2.5% of train observations)
Number of observations for testing: 36421
Train df shape:  (1627475, 4)


,latitude,longitude,species_id,subset
observation_id,,,,
10561949,45.705116,1.424622,241,train
10131188,45.146973,6.416794,101,train
10799362,46.783695,-2.072855,700,train


Test df shape:  (36421, 2)


,latitude,longitude
observation_id,,
10782781,43.601788,6.940195
10364138,46.241711,0.683586
10692017,45.181095,1.533459


Load environmental vectors, add coordinates and columns for the rgb/nir/alt/landcover patches, and fill nan values:

In [14]:
# Load the environmental vectors
df_env = pd.read_csv(DATA_PATH / "pre-extracted" / "environmental_vectors.csv", sep=";", index_col="observation_id")
display(df_env.head(3))

# Define the train, val and test set as np arrays
X_train = df_env.loc[obs_id_train].values
X_val = df_env.loc[obs_id_val].values
X_test = df_env.loc[obs_id_test].values

y_train_df = df_obs.loc[obs_id_train]["species_id"]
X_train_env_df = df_env.loc[obs_id_train]
X_train_coord_df = df_obs.loc[obs_id_train][["latitude","longitude"]]

y_val_df = df_obs.loc[obs_id_val]["species_id"]
X_val_env_df = df_env.loc[obs_id_val]
X_val_coord_df = df_obs.loc[obs_id_val][["latitude","longitude"]]

X_test_env_df = df_env.loc[obs_id_test]
X_test_coord_df = df_obs_test.loc[obs_id_test][["latitude","longitude"]]


# Merge environmental vectors with lon, lat coords and add cols for the patch info
X_train_df = pd.concat([X_train_env_df,X_train_coord_df], axis=1)
X_train_df[['mean_red','mean_green','mean_blue','mean_nir','mean_alt','mean_land']] = 0

X_val_df = pd.concat([X_val_env_df,X_val_coord_df], axis=1)
X_val_df[['mean_red','mean_green','mean_blue','mean_nir','mean_alt','mean_land']] = 0

X_test_df = pd.concat([X_test_env_df,X_test_coord_df], axis=1)
X_test_df[['mean_red','mean_green','mean_blue','mean_nir','mean_alt','mean_land']] = 0

# Replace nan values with np.min
X_train_df.fillna(np.finfo(np.float32).min, inplace=True)
X_val_df.fillna(np.finfo(np.float32).min, inplace=True)
X_test_df.fillna(np.finfo(np.float32).min, inplace=True)


# Display the heads to see the transformations
display(y_train_df.head(2))
# display(X_train_env_df.head(2)
# display(X_train_coord_df.head(2))
print("\n X_train : ", X_train_df.shape)
display(X_train_df.head(2))
print("\n X_val : ", X_val_df.shape)
display(X_val_df.head(2))
print("\n X_test : ", X_test_df.shape)
display(X_test_df.head(2))


# Load landcover metadata to use the patches
df_landcover_labels = pd.read_csv(DATA_PATH / "metadata" / "landcover_original_labels.csv", sep=";")
df_suggested_landcover_alignment = pd.read_csv(DATA_PATH / "metadata" / "landcover_suggested_alignment.csv", sep=";")
landcover_mapping = df_suggested_landcover_alignment["suggested_landcover_code"].values

display(df_landcover_labels.head(2))
display(df_suggested_landcover_alignment.head(2))

C:\Users\Louis\.conda\envs\SONDRA\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9,bio_10,...,bio_18,bio_19,bdticm,bldfie,cecsol,clyppt,orcdrc,phihox,sltppt,sndppt
observation_id,,,,,,,,,,,,,,,,,,,,,
10000000,1.420833,6.908333,29.272598,614.1493,15.1,-8.5,23.600000,-1.000000,9.183333,9.466667,...,248.0,358.0,2082.0,988.0,29.0,13.0,63.0,62.0,34.0,53.0
10000001,8.837500,9.858334,37.771393,586.8139,23.8,-2.3,26.099998,6.016667,16.383333,16.383333,...,226.0,288.0,1816.0,1142.0,20.0,22.0,39.0,58.0,41.0,36.0
10000002,6.241667,8.350000,32.239384,632.8609,21.0,-4.9,25.900000,3.033333,14.200000,14.200000,...,268.0,317.0,1346.0,1075.0,29.0,22.0,54.0,59.0,40.0,38.0


observation_id
10561949    241
10131188    101
Name: species_id, dtype: int64


 X_train :  (1587395, 35)


,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9,bio_10,...,sltppt,sndppt,latitude,longitude,mean_red,mean_green,mean_blue,mean_nir,mean_alt,mean_land
observation_id,,,,,,,,,,,,,,,,,,,,,
10561949,11.229167,8.724999,37.286324,556.81506,24.5,1.1,23.4,8.033333,18.266666,18.266666,...,36.0,44.0,45.705116,1.424622,0.0,0.0,0.0,0.0,0.0,0.0
10131188,4.587500,9.058333,33.302696,664.60220,19.9,-7.3,27.2,1.416667,12.800000,12.833333,...,38.0,45.0,45.146973,6.416794,0.0,0.0,0.0,0.0,0.0,0.0


...

,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9,bio_10,...,sltppt,sndppt,latitude,longitude,mean_red,mean_green,mean_blue,mean_nir,mean_alt,mean_land
observation_id,,,,,,,,,,,,,,,,,,,,,
10297014,10.875000,8.600000,33.463036,643.27466,25.4,-0.3,25.699999,11.033333,6.866667,18.966667,...,41.0,38.0,46.927032,4.798213,0.0,0.0,0.0,0.0,0.0,0.0
10610548,10.966667,8.483334,33.267975,644.26260,25.4,-0.1,25.500000,11.133333,6.950000,19.033333,...,43.0,39.0,46.865562,4.914852,0.0,0.0,0.0,0.0,0.0,0.0



 X_test :  (36421, 35)


,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9,bio_10,...,sltppt,sndppt,latitude,longitude,mean_red,mean_green,mean_blue,mean_nir,mean_alt,mean_land
observation_id,,,,,,,,,,,,,,,,,,,,,
10782781,14.837500,8.175,34.639830,562.34546,27.5,3.9,23.6,11.983334,21.983334,22.033333,...,34.0,43.0,43.601788,6.940195,0.0,0.0,0.0,0.0,0.0,0.0
10364138,11.854167,9.075,38.453392,551.89530,25.4,1.8,23.6,6.100000,18.833334,18.833334,...,37.0,43.0,46.241711,0.683586,0.0,0.0,0.0,0.0,0.0,0.0


,landcover_code,original_landcover_code,landcover_label
0,0,0,Missing Data
1,1,11,Annual Summer Crops


,landcover_code,suggested_landcover_code,suggested_landcover_label
0,0,0,Missing Data
1,1,11,Cultivated Crops


These functions will help perform the training and predictions with batches to not overflow memory:

In [15]:
def add_patch_info(X_df, landcover_mapping, DATA_PATH):

    for obs, values in X_df.iterrows():
            patch = load_patch(obs, DATA_PATH, landcover_mapping=landcover_mapping)
            rgb, nir, alt, land = patch[0], patch[1], patch[2], patch[3]

            values['mean_red'] = np.mean(rgb[:,:,0])
            values['mean_green'] = np.mean(rgb[:,:,1])
            values['mean_blue'] = np.mean(rgb[:,:,2])
            values['mean_nir'] = np.mean(nir)
            values['mean_alt'] = np.mean(alt)
            values['mean_land'] = round(np.mean(land))

            X_df.loc[obs] = values


# Define a batch_fit function to avoid memory overflow
# def batch_fit(est, X_df, Y, obs_id, batch_size=1024):
    
#     for i in range(0, len(X_df), batch_size):
#         obs_id_batch = obs_id[i:i+batch_size]
#         X_batch = X_df.loc[obs_id_batch, :]

#         add_patch_info(X_batch, DATA_PATH=DATA_PATH, landcover_mapping=landcover_mapping)
        
#         X_batch = X_batch.values
#         Y_batch = Y[i:i+batch_size]
#         est.fit(X_batch, Y_batch)
        
#         if (i/batch_size)%10 == 0:
#             print("Fitting : " + str(100*(i+1)/len(X_df)) + "% completed")

            

# We can finally compute the top 30 error rate on the val set
def predict_func(est, X):
    y_score = est.predict_proba(X)
    s_pred = predict_top_30_set(y_score)
    return s_pred            


# We define a batch predictor to take care of the memory
# as there are more than 17k classes
def batch_predict(predict_func, est, X_df, obs_id, batch_size=1024):
    res = predict_func(est, X_df.head(1).values)
    n_samples, n_outputs, dtype = X_df.shape[0], res.shape[1], res.dtype
    
    preds = np.empty((n_samples, n_outputs), dtype=dtype)
    print(preds.shape)
    
    for i in range(0, len(X_df), batch_size):
        obs_id_batch = obs_id[i:i+batch_size]
        X_batch = X_df.loc[obs_id_batch, :]
        
        add_patch_info(X_batch, DATA_PATH=DATA_PATH, landcover_mapping=landcover_mapping)
        
        X_batch = X_batch.values
        
        preds[i:i+batch_size] = predict_func(est, X_batch)

        if (i/batch_size)%10 == 0:
            print("Prediction : " + str(100*(i+1)/len(X_df)) + "% completed")
            
    return preds

## RF call, fit and prediction

In [16]:
# n_train = 100000
# n_val = 1000
# n_test = 1000

# # Reduce training set
# obs_id_train = obs_id_train[:n_train]
# X_train_df = X_train_df.loc[obs_id_train,:]
# y_train = y_train[:n_train]

# # Reduce training set
# obs_id_val = obs_id_val[:n_val]
# X_val_df = X_val_df.loc[obs_id_val,:]
# y_val = y_val[:n_val]

# # Reduce training set
# obs_id_test = obs_id_test[:n_test]
# X_test_df = X_test_df.loc[obs_id_test,:]


In [18]:
# Call a RF classifier, fit it on trainin set
from sklearn.ensemble import RandomForestClassifier
est = RandomForestClassifier(n_estimators=50, max_depth=12, n_jobs=-1, verbose=1)   
            
print("***** Fitting started *****")
est.fit(X_train, y_train)
print("***** Fitting successful *****\n")


# Validation
print("***** Batch predict started *****")
s_val = batch_predict(predict_func, est, X_val_df, obs_id_val)
print("***** Batch predict successful *****\n")

score_val = top_k_error_rate_from_sets(y_val, s_val)
print("Top-30 error rate: {:.1%}".format(score_val))


# Compute baseline on the test set
print("***** Batch predict test started *****")
s_pred = batch_predict(predict_func, est, X_test_df, obs_id_test)
print("***** Batch predict test successful *****\n")

# Generate the submission file
generate_submission_file(SUBMISSION_PATH / "random_forest_on_environmental_vectors.csv", df_obs_test.index[:s_pred.shape[0]], s_pred)

***** Fitting started *****


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished


Fitting : 6.299629266817648e-05% completed


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


Fitting : 0.6451450332147953% completed


...

Fitting : 3.870555217825431% completed


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


Fitting : 4.515637254747558% completed


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished


KeyboardInterrupt: 